In [ ]:
# Import basic libraries
import numpy as np
import pandas as pd
import pickle
import os.path
import matplotlib.pyplot as plt
from joblib import Memory
from joblib import Parallel, delayed
import joblib
from scipy import stats
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# import mne
import mne
from mne.datasets import sample
from mne.decoding import (SlidingEstimator, GeneralizingEstimator, Scaler,
                          cross_val_multiscore, LinearModel, get_coef,
                          Vectorizer, CSP)
from mne import (read_epochs, combine_evoked, write_evokeds)
from mne.stats import bonferroni_correction, fdr_correction
from mne import io

In [ ]:
# load the basic folder names and the location of the eeg datasets
folder_name_eeg = "E:\\SUMO_further_data_pack_zx\\N2pc_IEM\\new_results\\eeg_before_IEM\\"
individual_name_eeg = 'SUMO_0102_before_iem_stim.set'
eeg_file_name = str(folder_name_eeg + individual_name_eeg)

# load the eeg files to the mne 
eeg = mne.read_epochs_eeglab(eeg_file_name)

In [ ]:
# have the label as the test file
df = pd.read_csv("cue102_label.csv")
df2 = df.copy()

# change from 1-len(label) to 0 - len(label)-1(from MATLAB to python)
df2.epoch = df.epoch - 1
df2.bin = df.bin - 2

# sort the row by epoch number for later decoding
# df2 = df2.sort_values('epoch')

# # extract eeg data from epoch number by different bins
# eeg_bin0 = eeg[df2[df2.bin == 0].epoch].get_data()
# print("The number of trials for SUMO 102 in cue1 condition in bin 0 is: " + str(len(eeg_bin0)))
# eeg_bin1 = eeg[df2[df2.bin == 1].epoch].get_data()
# print("The number of trials for SUMO 102 in cue1 condition in bin 1 is: " + str(len(eeg_bin1)))
# eeg_bin2 = eeg[df2[df2.bin == 2].epoch].get_data()
# print("The number of trials for SUMO 102 in cue1 condition in bin 2 is: " + str(len(eeg_bin2)))
# eeg_bin3 = eeg[df2[df2.bin == 3].epoch].get_data()
# print("The number of trials for SUMO 102 in cue1 condition in bin 3 is: " + str(len(eeg_bin3)))
# eeg_bin4 = eeg[df2[df2.bin == 4].epoch].get_data()
# print("The number of trials for SUMO 102 in cue1 condition in bin 4 is: " + str(len(eeg_bin4)))
# eeg_bin5 = eeg[df2[df2.bin == 5].epoch].get_data()
# print("The number of trials for SUMO 102 in cue1 condition in bin 5 is: " + str(len(eeg_bin5)))

In [ ]:
number_of_cpu = joblib.cpu_count()
clf = make_pipeline(Scaler(eeg.info),
                    Vectorizer(),
                    LogisticRegression(solver='lbfgs'))
# CSP
csp = CSP(n_components=5, norm_trace=False, cov_est='concat', cov_method_params = dict)
clf_csp = make_pipeline(csp, LinearModel(LogisticRegression(solver='lbfgs')))
# Time_decoding
clf_1 = make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs'))
time_decode = SlidingEstimator(clf_1, n_jobs=number_of_cpu, scoring='roc_auc', verbose=True)

In [ ]:
score_mean = np.empty([6,1000])
for i in range(6):
    df3 = df2.copy()
    df3.loc[df3['bin'] != i, 'bin'] = 9
    eeg2 = eeg[df3.epoch].get_data()
    score = cross_val_multiscore (time_decode, eeg2, df3.bin, cv = 10, n_jobs=number_of_cpu)
    score_mean[i] = np.mean(score, axis=0)
    
p = np.mean(score_mean, axis=0)

In [ ]:
plt.figure(figsize=(15, 6), dpi=80)
plt.plot(eeg.times, p, 'b', label='Scene_post')
xmin, xmax = plt.xlim()
# plt.hlines(0.5, xmin, xmax, linestyle='--', colors='k',
#            label='chance', linewidth=2)
#plt.hlines(threshold_fdr_f_p, xmin, xmax, linestyle='--', colors='b',
           #label='p=0.05 (FDR)', linewidth=2)
plt.legend()
plt.xlabel("Time (sec)")
plt.ylabel("AUC")
# plt.savefig("E4.pdf", format="pdf", bbox_inches="tight")
plt.show()